PROBLEM 2

In [3]:
import pandas as pd
user_ratings_nam = ["user_id", "item_id", "rating", "timestamp"]
user_ratings = pd.read_csv('u.data', sep='\t', header = None, names = user_ratings_nam,encoding='latin-1')
movie_nam = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep = '|', header = None, names = movie_nam,encoding='latin-1')
user_nam = ["user_id", "age", "gender", "occupation", "zipcode"]
users = pd.read_csv('u.user', sep = '|', header = None, names = user_nam,encoding='latin-1')
users.head()

,user_id,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
def profile_user():
    rec = 0
    genres = movie_nam[6:]
    genre_ratings=[]
    genre_counts=[]
    user_ids = [200, 15]
    profile= user_ids
    lis=[]

    for i in user_ids:
        profile = [i]
        genre_ratings = [0]*len(genres)
        genre_counts = [0]*len(genres)
        per_user_rating=user_ratings[user_ratings['user_id']==i]
        movie_ids = per_user_rating['item_id']
        movie_ratings = per_user_rating['rating']
        for i, r in zip(movie_ids, movie_ratings):
            movie = movies[movies['movie id']==i]
            for k, j in enumerate(genres):
                if movie[j].iloc[0] == 1:
                    genre_ratings[k] += r
                    genre_counts[k] += 1
        rec += 1
        genre_ratings = [round(i/j, 2) if j != 0 else 0 for i,j in zip(genre_ratings, genre_counts)]
        profile.extend(genre_ratings)
        lis.append(profile)
        
    return  lis

In [7]:
columns =['User_id','Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
p=profile_user()

p=pd.DataFrame(p)
p.columns=columns
print(p)

   User_id  Action  Adventure  Animation  ...  Sci-Fi  Thriller   War  Western
0      200    4.20       4.12       4.12  ...    4.37      4.02  4.29      4.0
1       15    2.68       3.00       1.00  ...    2.67      2.36  3.40      0.0

[2 rows x 19 columns]


Cosine similarity between user15 ,user200 and movie 97

In [12]:
from scipy.spatial.distance import cosine
genres=movie_nam[6:]
data=user_ratings
user200=cosine(movies[movies['movie id']==95][genres].values[0], p[genres].values[0])
user15=cosine(movies[movies['movie id']==95][genres].values[0], p[genres].values[1])
print("cosine similarity between user 15 and movie 97 :",user15)
print("cosine similarity between user 200 and movie 97 :",user200)

cosine similarity between user 15 and movie 97 : 0.6427084353986308
cosine similarity between user 200 and movie 97 : 0.5338775894666912


Recommend movie 97 user 15

Problem 3

In [10]:
user_ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
import numpy as np
data_pivot = pd.pivot_table(data,values='rating',index='item_id',columns='user_id')
data_pivot.fillna( 0, inplace = True )
from sklearn.metrics.pairwise import cosine_similarity

a_1 = np.array(data_pivot.iloc[567]).reshape(1, -1)  #567 is the row number of movie 'Speed'

 
# calculate the similartity of this movie with the others in the list
score_col = cosine_similarity(data_pivot, a_1).reshape(-1)


# form a data frame of similar movies 
dictdf = {'collaborative': score_col } 
similar = pd.DataFrame(dictdf, index = data_pivot.index )
 
#sort it on the basis of collaborative score
similar.sort_values('collaborative', ascending=False, inplace=True)
#similar[['collaborative']][1:].head(10)

In [15]:
data_tmp=pd.merge(users,user_ratings,on='user_id')

data_piv = pd.pivot_table(data_tmp,values='rating',index='user_id',columns='item_id')
bins = [18, 30, 40, 50, 60, 70, 120]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70+']
data_tmp['agerange'] = pd.cut(data_tmp.age, bins, labels = labels,include_lowest = True)

data_summary=data_tmp.groupby('agerange',as_index=True)['rating'].count()
data_tmp.head(7)

,user_id,age,gender,occupation,zipcode,item_id,rating,timestamp,agerange
0,1,24,M,technician,85711,61,4,878542420,18-29
1,1,24,M,technician,85711,189,3,888732928,18-29
2,1,24,M,technician,85711,33,4,878542699,18-29
3,1,24,M,technician,85711,160,4,875072547,18-29
4,1,24,M,technician,85711,20,4,887431883,18-29
5,1,24,M,technician,85711,202,5,875072442,18-29
6,1,24,M,technician,85711,171,5,889751711,18-29


From the above table users who like speed belong to 18-29 age group